In [1]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("data/train_filled.parquet")

# 讀取top50 features
n_top = 80
save_dir = "model_output/model_parameter/"
df = pd.read_csv("model_output/all_features_xgb/v1/v2_2/model_importance/common_features_with_ranks.csv")
top50_features = df["feature"].head(n_top).tolist()

# 你要保留的欄位 (Top50 + target + group id)
cols_to_keep = top50_features + ["selected", "ranker_id"]

# 只保留這些欄位
train_filled = train_filled.select(cols_to_keep)


print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 82 欄位: ['total_is_min_transfers', 'total_num_transfers_rank', 'price_per_duration_rank', 'price_from_median_zscore', 'legs0_num_transfers_rank', 'legs0_segments0_flightNumber', 'price_percentile', 'legs0_mean_cabin', 'price_per_duration', 'companyID', 'price_minus_fee_rank', 'legs0_max_duration_cabin', 'price_per_fee_rank', 'days_before_departure', 'legs0_segments0_baggageAllowance_quantity', 'totalPrice_rank', 'has_access_tp', 'both_legs_carrier_all_same', 'pricingInfo_isAccessTP', 'legs0_arrivalAt_hour', 'legs0_departureAt_hour', 'price_per_tax', 'legs1_weighted_mean_cabin', 'baggage_total', 'price_per_fee', 'total_weighted_mean_cabin', 'duration_ratio', 'legs1_segments0_flightNumber', 'legs0_segments0_cabinClass', 'log_price', 'legs0_segments0_seatsAvailable', 'legs1_segments0_baggageAllowance_quantity', 'legs0_segments0_marketingCarrier_code', 'legs1_departureAt_hour', 'price_minus_fee', 'legs1_arrivalAt_hour', 'has_baggage', 'isVip', 'corporateTariffCode', 'legs0_

In [2]:
exclude_cols = [
    'Id', 'ranker_id', 'selected'
]

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

Using 80 features


In [6]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]
# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 先抽一部分ranker_id
max_rankers = 20000  # 先抽500個id
sample_size = min(max_rankers, len(unique_rankers))
np.random.seed(42)
sampled_rankers = np.random.choice(unique_rankers, size=sample_size, replace=False)
np.random.shuffle(sampled_rankers)

# 切8:2
n_train = int(0.8 * len(sampled_rankers))
train_rankers = set(sampled_rankers[:n_train])
val_rankers = set(sampled_rankers[n_train:])

# 用 Polars 過濾 train/val
# train mask
is_train = groups.select(
    pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train")
)

# val mask
is_val = groups.select(
    pl.col("ranker_id").is_in(list(val_rankers)).alias("is_val")
)

# concat mask
X_with_train = X.with_columns(is_train)
X_with_val = X.with_columns(is_val)

y_with_train = y.with_columns(is_train)
y_with_val = y.with_columns(is_val)

groups_with_train = groups.with_columns(is_train)
groups_with_val = groups.with_columns(is_val)

# split
X_train_df = X_with_train.filter(pl.col("is_train"))
X_val_df = X_with_val.filter(pl.col("is_val"))

y_train_df = y_with_train.filter(pl.col("is_train"))
y_val_df = y_with_val.filter(pl.col("is_val"))

groups_train_df = groups_with_train.filter(pl.col("is_train"))
groups_val_df = groups_with_val.filter(pl.col("is_val"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_val").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_val").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_val").to_numpy().flatten()
# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_val")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)


In [7]:
print(f"train rows: {X_train_df.shape[0]}")
print(f"val rows: {X_val_df.shape[0]}")


train rows: 2778295
val rows: 710419


In [8]:
import xgboost as xgb
import gc

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
    feature_names = feature_cols,
)
dtrain.set_group(group_sizes_train)
del X_train_np, y_train_np, group_sizes_train
gc.collect()
dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
    feature_names = feature_cols,
)
dval.set_group(group_sizes_val)
del X_val_np, group_sizes_val
gc.collect()


7

In [9]:
import itertools
import xgboost as xgb
import numpy as np
import os
import json

# 定義超參數空間
param_grid = {
    'max_depth': [12, 13,14],
    'min_child_weight': [1,3],
    'learning_rate': [0.05, 0.1,0.2],
    'subsample': [1.0,0.9, 0.8],
    'colsample_bytree': [1.0,0.9, 0.8],
    'lambda': [0,1],
}

# 所有組合
all_combinations = list(itertools.product(*param_grid.values()))
print(f"共 {len(all_combinations)} 組超參數")

best_score = -np.inf
best_params = None
best_model = None

for i, combo in enumerate(all_combinations, 1):
    # 組成dict
    params = dict(zip(param_grid.keys(), combo))
    params.update({
        'objective': 'rank:pairwise',
        'eval_metric': 'ndcg@3',
        'seed': 42,
        'n_jobs': -1,
    })

    print(f"\n🔹 第 {i}/{len(all_combinations)} 組: {params}")

    evals_result = {}

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=[(dtrain, "train"), (dval, "val")],
        evals_result=evals_result,   # 這裡要加
        early_stopping_rounds=30,
        verbose_eval=100
    )

    # 拿最後一個 eval result
    best_ndcg = evals_result["val"]["ndcg@3"][-1]
    print(f"val ndcg@3 = {best_ndcg:.5f}")

    if best_ndcg > best_score:
        best_score = best_ndcg
        best_params = params
        best_model = model
        print("✅ 更新最佳模型")

print("\n🏆 最佳組合:")
print(best_params)
print(f"Best val ndcg@3 = {best_score:.5f}")
# 儲存最佳參數
os.makedirs(save_dir, exist_ok=True)
param_path = os.path.join(save_dir, "best_xgb_params.json")

with open(param_path, "w") as f:
    json.dump(best_params, f, indent=4)

print(f"✅ 已儲存最佳參數至 {param_path}")


共 324 組超參數

🔹 第 1/324 組: {'max_depth': 12, 'min_child_weight': 1, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'lambda': 0, 'objective': 'rank:pairwise', 'eval_metric': 'ndcg@3', 'seed': 42, 'n_jobs': -1}
[0]	train-ndcg@3:0.37319	val-ndcg@3:0.32753
[100]	train-ndcg@3:0.80713	val-ndcg@3:0.42770
[200]	train-ndcg@3:0.86618	val-ndcg@3:0.43674
[212]	train-ndcg@3:0.87313	val-ndcg@3:0.43679
val ndcg@3 = 0.43679
✅ 更新最佳模型

🔹 第 2/324 組: {'max_depth': 12, 'min_child_weight': 1, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 1.0, 'lambda': 1, 'objective': 'rank:pairwise', 'eval_metric': 'ndcg@3', 'seed': 42, 'n_jobs': -1}
[0]	train-ndcg@3:0.37133	val-ndcg@3:0.33019
[100]	train-ndcg@3:0.81255	val-ndcg@3:0.42901
[200]	train-ndcg@3:0.85764	val-ndcg@3:0.43963
[300]	train-ndcg@3:0.95683	val-ndcg@3:0.44862
[400]	train-ndcg@3:0.98972	val-ndcg@3:0.45581
[492]	train-ndcg@3:0.99587	val-ndcg@3:0.45873
val ndcg@3 = 0.45841
✅ 更新最佳模型

🔹 第 3/324 組: {'max_depth': 12, 'min_child_